In [1]:
from selenium import webdriver
import time
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium.webdriver.common.keys import Keys
from multiprocessing import Process

# 모바일 게임 랭킹 데이터베이스 수집

In [2]:
driver = webdriver.Chrome('./chromedriver')

url = 'http://www.mobileindex.com/'
driver.get(url)

soup = BeautifulSoup(driver.page_source,'html.parser')

In [3]:
#-------------------------------------------------------
#이 작업 이전에 chromedriver를 직접 조작하여 게임정보를 검색하였음.
table = soup.select('body > div.container.app-db > div.search-result > table > tbody > tr > td')

print("항목의 개수\n=", len(table))

항목의 개수
= 0


In [22]:
print("첫 게임에 대한 정보\n")
print(*table[:8],sep='\n')

첫 게임에 대한 정보

<td class="text-center">1</td>
<td><img alt="" src="http://upload.gevolution.co.kr/appicon/s0014000/gevolution_00014001_icon_100.png" width="40"/></td>
<td><a href="/app/?svc=14001&amp;t=schlist">클래시 오브 클랜</a></td>
<td>Supercell</td>
<td class="text-center">전략</td>
<td class="text-center">13.10.01</td>
<td class="text-center">1</td>
<td class="text-center">34</td>


In [31]:
first_element = table[:8]

print("Index   :",first_element[0].get_text())
print("Title   :",first_element[2].get_text())
print("Pub     :",first_element[3].get_text())
print("Genre   :",first_element[4].get_text())
print("Birth   :",first_element[5].get_text())
print("Max_rank:",first_element[6].get_text())
print("Curr_rank:",first_element[7].get_text())
print("link:   ",'http://www.mobileindex.com'+re.search('href="(.*?);(.*)"',str(first_element[2])).group(1))

Index   : 1
Title   : 클래시 오브 클랜
Pub     : Supercell
Genre   : 전략
Birth   : 13.10.01
Max_rank: 1
Curr_rank: 34
link:    http://www.mobileindex.com/app/?svc=14001&amp


# 모바일 게임 데이터프레임 구축

In [44]:
games = [table[8*x+2:8*x+8] for x in range(len(table)//8)]
DF_list = []

for x in range(len(games)):
    link = 'http://www.mobileindex.com'+re.search('href="(.*?);(.*)"',str(games[x][0])).group(1)
    elements = [e.get_text() for e in games[x]]
    elements.append(link)
    DF_list.append(elements)
    
columns = ['게임이름','퍼블리셔','장르','제조일자','최고순위','현재순위','링크']
DF = pd.DataFrame(DF_list, columns=columns)

In [48]:
DF['최고순위'] = DF.최고순위.apply(lambda x : int(x))
DF['현재순위'] = DF.현재순위.apply(lambda x : int(x))

DF = DF.set_index('게임이름')

In [55]:
DF.head(5)

,퍼블리셔,장르,제조일자,최고순위,현재순위,링크
게임이름,,,,,,
클래시 오브 클랜,Supercell,전략,13.10.01,1,34,http://www.mobileindex.com/app/?svc=14001&amp
모두의마블 for kakao,Netmarble,보드,13.06.12,1,7,http://www.mobileindex.com/app/?svc=13929&amp
몬스터 길들이기 for kakao,Netmarble,롤플레잉,13.08.14,1,93,http://www.mobileindex.com/app/?svc=13922&amp
"피망 포커: 카지노 로얄 (7포커, 로우바둑이, 하이로우)",NEOWIZ corp,카지노,13.08.28,1,15,http://www.mobileindex.com/app/?svc=14082&amp
애니팡2,"SUNDAYTOZ, INC",퍼즐,14.01.15,1,70,http://www.mobileindex.com/app/?svc=13919&amp


In [2]:
#DF.to_excel('게임순위데이터_old.xlsx')

DF = pd.read_excel('게임순위데이터_old.xlsx')

# 플레이스토어 링크 수집

In [3]:
def store_link_crawl(GAME_NAMES,SITE_LINKS):

    url_and_title = []

    for x in range(len(GAME_NAMES)):    
        game_url = SITE_LINKS[x]
        temporal = webdriver.Chrome('./chromedriver')
        temporal.get(game_url)
        game_soup = BeautifulSoup(temporal.page_source,'html.parser')
        selected = game_soup.select('div.container.app-detail > div.app-info > table > tbody > tr > td')[4]
        temporal.quit()
        play_store_url = re.search('href="(.*?)"',str(selected)).group(1)
        url_and_title.append([GAME_NAMES[x],play_store_url])
        print("%s번째 플레이스토어 링크 획득!"%(x))
        
    return url_and_title

In [4]:
url_and_title = store_link_crawl(DF.게임이름,DF.링크)

0번째 플레이스토어 링크 획득!
1번째 플레이스토어 링크 획득!
2번째 플레이스토어 링크 획득!
3번째 플레이스토어 링크 획득!
4번째 플레이스토어 링크 획득!
5번째 플레이스토어 링크 획득!
6번째 플레이스토어 링크 획득!
7번째 플레이스토어 링크 획득!
8번째 플레이스토어 링크 획득!
9번째 플레이스토어 링크 획득!
10번째 플레이스토어 링크 획득!
11번째 플레이스토어 링크 획득!
12번째 플레이스토어 링크 획득!
13번째 플레이스토어 링크 획득!
14번째 플레이스토어 링크 획득!
15번째 플레이스토어 링크 획득!
16번째 플레이스토어 링크 획득!
17번째 플레이스토어 링크 획득!
18번째 플레이스토어 링크 획득!
19번째 플레이스토어 링크 획득!
20번째 플레이스토어 링크 획득!
21번째 플레이스토어 링크 획득!
22번째 플레이스토어 링크 획득!
23번째 플레이스토어 링크 획득!
24번째 플레이스토어 링크 획득!
25번째 플레이스토어 링크 획득!
26번째 플레이스토어 링크 획득!
27번째 플레이스토어 링크 획득!
28번째 플레이스토어 링크 획득!
29번째 플레이스토어 링크 획득!
30번째 플레이스토어 링크 획득!
31번째 플레이스토어 링크 획득!
32번째 플레이스토어 링크 획득!
33번째 플레이스토어 링크 획득!
34번째 플레이스토어 링크 획득!
35번째 플레이스토어 링크 획득!
36번째 플레이스토어 링크 획득!
37번째 플레이스토어 링크 획득!
38번째 플레이스토어 링크 획득!
39번째 플레이스토어 링크 획득!
40번째 플레이스토어 링크 획득!
41번째 플레이스토어 링크 획득!
42번째 플레이스토어 링크 획득!
43번째 플레이스토어 링크 획득!
44번째 플레이스토어 링크 획득!
45번째 플레이스토어 링크 획득!
46번째 플레이스토어 링크 획득!
47번째 플레이스토어 링크 획득!
48번째 플레이스토어 링크 획득!
49번째 플레이스토어 링크 획득!
50번째 플레이스토어 링크 획득!
51번째 플레이스토어 링크 획득!
52번째 플레이스토어 링크 획득!
53번

# 플레이스토어 리뷰 수집

In [11]:
def get_score(string):
    try:
        return re.search('별표 5개 만점에 ([0-9])개를 받았습니다.',string).group(1)
    except:
        return 0
    
def get_time(string):
    try:
        return re.search('class="p2TkOb">(.*?)</span>',string).group(1)
    except:
        return -1
    
def get_review(string):
    try:
        return re.search('jsname="bN97Pc">(.*?)</span>',string).group(1)
    except:
        return ''
    
def login(driver2,gmail,password):
    
    login_url = driver2.find_element_by_css_selector('.gb_Ee.gb_Aa.gb_Hb').get_attribute('href')

    driver2.get(login_url)

    driver2.find_element_by_css_selector('.dMArKd.bxPAYd.k6Zj8d > content > div > div > div > div > div > input').send_keys(gmail)

    driver2.find_element_by_id('identifierNext').click()

    time.sleep(1)

    driver2.find_element_by_css_selector('.whsOnd.zHQkBf').send_keys(password)

    driver2.find_element_by_id('passwordNext').click()

    time.sleep(3)

In [6]:
def crawling(url_list,game_names,gmail,password,start,bin):

    driver2 = webdriver.Chrome('./chromedriver')
    play_store_url = url_list[0]
    driver2.get(play_store_url+'&showAllReviews=true')
    login(driver2,gmail,password)
    review_list = []
    for x in range(start,start+bin):
        game_name = game_names[x]
        if game_name == 'MLB 9이닝스 18':
            continue
        play_store_url = url_list[x]
        # .BHMmbe는 날마다 달라지는거같음.
        whole_average_score = BeautifulSoup(requests.get(play_store_url).text,'html.parser').select_one('.BHMmbe').get_text()
        driver2.get(play_store_url+'&showAllReviews=true')

        time.sleep(2)

        driver2.find_element_by_css_selector('.MocG8c.XPZEHc.LMgvRb.KKjvXb').click()

        time.sleep(2)

        driver2.find_element_by_css_selector('.OA0qNb.ncFHed > div > content').click()

        time.sleep(1)

        current = time.time()

        driver2.maximize_window()

        while (time.time() - current) <= 600:

            for y in range(20):

                body = driver2.find_element_by_id('yDmH0d')

                body.send_keys(Keys.PAGE_DOWN)

                time.sleep(0.2)

            try:

                driver2.find_element_by_class_name('CwaK9').click()
                time.sleep(0.5)

            except:

                pass


        try:
            soup = BeautifulSoup(driver2.page_source,'html.parser')
        except:
            print("%s 게임에서 오류가 발생하여 중단하고 다음 게임의 데이터를 수집합니다."%(game_name))
            continue

        selected = soup.select('.W4P4ne > div > div > div')

        average = 0

        for y in range(len(selected)-1):

            string = str(selected[y])

            score = int(get_score(string))
            average += score

            creation_time = get_time(string)

            review = get_review(string)

            review_list.append([score,creation_time,review,game_name])

        print("----------------------------------------------------")
        print("Summary\n")
        print("게임 이름        ",game_name)
        print("리뷰의 개수      ",len(selected))
        print("가장 오래된 리뷰  ",creation_time)
        print("수집한 리뷰의 평점",average/len(selected))
        print("전체 리뷰의 평점  ",whole_average_score)
    
    review_df = pd.DataFrame(review_list)
    review_df.to_excel('review_%s.xlsx'%(start))
        
               

In [18]:
links = [url_and_title[x][1] for x in range(len(url_and_title))]
game_names = [url_and_title[x][0] for x in range(len(url_and_title))]
numbers = [0,30,60,90,120,150]
procs = []
bin = 30

for number in numbers:
    proc = Process(target=crawling,args=(links,game_names,gmail,password,number,bin,))
    procs.append(proc)
    proc.start()
    
for proc in procs:
    proc.join()

Process Process-31:
Traceback (most recent call last):


KeyboardInterrupt: 

  File "/home/dongchan/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse(buffering=True)
TypeError: getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

Process Process-35:
Process Process-36:
Traceback (most recent call last):
Process Process-33:
Traceback (most recent call last):
Process Process-32:
Traceback (most recent call last):
  File "/home/dongchan/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process Process-34:
  File "/home/dongchan/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dongchan/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse(buffering=True)
  File "<ipython-input-6-6fbe4576e8ad>", line 43, in cra

  File "/home/dongchan/anaconda3/lib/python3.6/site-packages/urllib3/request.py", line 70, in request
    **urlopen_kw)
  File "/home/dongchan/anaconda3/lib/python3.6/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/home/dongchan/anaconda3/lib/python3.6/site-packages/urllib3/request.py", line 148, in request_encode_body
    return self.urlopen(method, url, **extra_kw)
  File "/home/dongchan/anaconda3/lib/python3.6/site-packages/urllib3/poolmanager.py", line 321, in urlopen
    response = conn.urlopen(method, u.request_uri, **kw)
  File "/home/dongchan/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 639, in urlopen
    _stacktrace=sys.exc_info()[2])
  File "/home/dongchan/anaconda3/lib/python3.6/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/home/dongchan/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=c